In [1]:
import os
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split as SPLIT
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
df = pd.read_csv('combined_cleaned.csv')



In [3]:
# modify birthyear-> 0-1 on a float scale
birthyears = df.birthyear.unique()
_min = min(birthyears)
_max = max(birthyears)

def normalize_birthyear(year):
    return (year-_min)/(_max-_min)

birthyear_labels = df.birthyear.apply(normalize_birthyear)
birthyear_labels = birthyear_labels.values
birthyear_labels

array([0.64705882, 0.30882353, 0.72058824, ..., 0.45588235, 0.73529412,
       0.58823529])

In [10]:
labels_to_onehot = ['fame', 'gender', 'occupation']

from sklearn.preprocessing import LabelBinarizer

labels = {}

for label in labels_to_onehot:
    unique_classes = len(df[label].unique())
    print('{} unique classes in {}'.format(unique_classes, label))
    labels_nd = df[label].values
    labels[label] = labels_nd
    #encoder = LabelBinarizer()
    #labels[label] = encoder.fit_transform(labels_nd)
    labels[label] = pd.get_dummies(labels_nd)

2 unique classes in fame
3 unique classes in gender
8 unique classes in occupation


In [17]:
fame_labels = labels['fame']
gender_labels = labels['gender']
occ_labels = labels['occupation']

In [18]:
#fame_labels = to_categorical(fame_labels)
#gender_labels = to_categorical(gender_labels)
#occ_labels = to_categorical(occ_labels)

print(fame_labels.shape)
print(gender_labels.shape)
print(occ_labels.shape)

(6996, 2)
(6996, 3)
(6996, 8)


In [20]:
from keras.preprocessing.text import Tokenizer
vocab_size = 15000
 
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(df.text)
 
x_train = tokenizer.texts_to_matrix(df.text, mode='tfidf')

In [21]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokenizer.pickle', 'rb') as handle:
#    tokenizer = pickle.load(handle)

In [35]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

fame_labels.shape[1]

2

In [38]:
shape = (vocab_size,)
input_layer = Input(shape)
# before splitting categories
branch_layer = Dense(units=128, activation='relu')(input_layer)
# birthyear
_ = Dense(units=128, activation='relu')(branch_layer)
birthyear_out = Dense(units=1, activation='sigmoid', name='birthyear_out')(_)

# fame
_ = Dense(units=128, activation='relu')(branch_layer)
fame_out = Dense(units=fame_labels.shape[1], activation='softmax', name='fame_out')(_)

# gender
_ = Dense(units=128, activation='relu')(branch_layer)
gender_out = Dense(units=gender_labels.shape[1], activation='softmax', name='gender_out')(_)

# occupation
_ = Dense(units=128, activation='relu')(branch_layer)
occ_out = Dense(units=occ_labels.shape[1], activation='softmax', name='occ_out')(_)

model = Model(input=input_layer, outputs=[birthyear_out, fame_out, gender_out, occ_out])
#model = Model(input=input_layer, outputs=fame_out)
loss_fn = 'categorical_crossentropy'

model.compile(optimizer='adam',
              loss={'birthyear_out': 'mse', 'fame_out': loss_fn, 'gender_out': loss_fn, 'occ_out': loss_fn},
              metrics={'birthyear_out': 'mae', 'fame_out': 'accuracy', 'gender_out': 'accuracy', 'occ_out': 'accuracy'}
             )

#model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 15000)        0                                            
__________________________________________________________________________________________________
dense_24 (Dense)                (None, 128)          1920128     input_8[0][0]                    
__________________________________________________________________________________________________
dense_25 (Dense)                (None, 128)          16512       dense_24[0][0]                   
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 128)          16512       dense_24[0][0]                   
__________________________________________________________________________________________________
dense_27 (

C:\Users\Tollef\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [ ]:
model.fit(x_train, [df.birthyear.values, fame_labels, gender_labels, occ_labels], epochs=2, batch_size=32)




Instructions for updating:
Use tf.cast instead.
Epoch 1/2
 416/6996 [>.............................] - ETA: 33s - loss: 3905906.8846 - birthyear_out_loss: 3905900.2115 - fame_out_loss: 1.3021 - gender_out_loss: 1.8349 - occ_out_loss: 3.5745 - birthyear_out_mean_absolute_error: 1976.2872 - fame_out_acc: 0.5673 - gender_out_acc: 0.6034 - occ_out_acc: 0.3678

In [ ]:
model.fit(x_train, _fame, epochs=2, batch_size=32)